In [1]:
import sys
import os

# Add the project root to the search path
sys.path.append(os.path.abspath(".."))

# Import from src
from src import *


In [2]:
# Generate random max-2SAT instance

N = 3    # number of variables
M = N*2   # number of clauses

instance = generate_max_2_sat(N, M)
print("instance in cnf form")
instance

instance in cnf form


[(-1, -3), (3, -2), (1, -2), (1, -1), (1, 2), (3, -3)]

In [ ]:
# Get the exact solution using rc2 solver

sol, sol_nvc = rc2_solver(N, instance)
print("Solution :", sol)
print("Number of violated clauses:", sol_nvc)

Solution : [ 1 -1 -1]
Number of violated clauses: 0


In [4]:
# using SDP on the instance

Y =SDP_max_sat(N, instance, "Mosek")

In [5]:
# rounding the SDP solution

NVC = rounding_87(Y, N, instance)

In [6]:
print("Average approx ratio :",(M-np.mean(NVC))/(M-sol_nvc))
print("Best approx ratio    :",(M-np.min(NVC))/(M-sol_nvc))
print("Worst approx ratio   :",(M-np.max(NVC))/(M-sol_nvc))

Average approx ratio : 1.0
Best approx ratio    : 1.0
Worst approx ratio   : 1.0


In [7]:
Y

array([[ 0.99999999,  0.99999999, -0.99999997, -0.99999998],
       [ 0.99999999,  1.        , -0.99999998, -0.99999999],
       [-0.99999997, -0.99999998,  0.99999999,  0.99999999],
       [-0.99999998, -0.99999999,  0.99999999,  1.        ]])

In [8]:
u = np.random.randn(N + 1)  # Normal vector for a random hyperplane
# x = np.sign(Y.value @ u)    # Project SDP solution to ±1 solution
# x = np.sign((sp.linalg.sqrtm(Y.value).real) @ u)    # Project SDP solution to ±1 solution
eigenvalues, eigenvectors = np.linalg.eigh(Y)
# Compute the square root
Sqrt_y = eigenvectors @ np.diag(np.sqrt(np.maximum(eigenvalues, 0))) @ eigenvectors.T
x = np.sign(Sqrt_y @ u)
solution = x[0] * x[1:]     # Extract solution in ±1 format
nvc = violated_clauses(instance, solution)  # Count violated clauses
Sqrt_y

array([[ 0.50007916,  0.50001321, -0.49993665, -0.49997096],
       [ 0.50001321,  0.50005288, -0.49997096, -0.49996294],
       [-0.49993665, -0.49997096,  0.50007916,  0.50001321],
       [-0.49997096, -0.49996294,  0.50001321,  0.50005288]])